In [1]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_rows', None)
#coaching_time_compare

Processing Fitbit data

In [157]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")

#coach data
coach_time = pd.read_excel("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\caoching_time.xlsx")

In [4]:
dailyactivity.rename(columns = {'ActivityDate':'date'}, inplace=True)
wear_time.rename(columns = {'Day':'date'}, inplace=True)
dailyFitbitActiveZoneMinutes.rename(columns = {'Date':'date'}, inplace=True)
dailyHRV.rename(columns = {'SleepDay':'date'}, inplace=True)
fitbitBreathingRate.rename(columns = {'SleepDay':'date'}, inplace = True)
fitbitSkinTemperature.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepDay.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepStageLogInfo.rename(columns = {'SleepDay':'date'}, inplace=True)
heartrate_15min.rename(columns = {'Time':'date'}, inplace=True)

In [ ]:
# transfer date
total_data = [wear_time, dailyactivity, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, 
              fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min]

for d in total_data:
    d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
    d['date'] = d['date'].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer heart_rate_15 min
heartrate_15min = heartrate_15min.groupby(by = ['Id','date']).mean().reset_index()

In [11]:
merged1 = pd.merge(wear_time, dailyactivity, on=['Id','date'], how='left')  # inner join only keeps common dates
merged2 = pd.merge(merged1, sleepDay, on=['Id','date'], how='left')
merged3 = pd.merge(merged2, sleepStageLogInfo, on=['Id','date'], how='left')
merged4 = pd.merge(merged3, heartrate_15min, on=['Id','date'], how='left')
merged5 = pd.merge(merged4, dailyFitbitActiveZoneMinutes, on=['Id','date'], how='left')
merged6 = pd.merge(merged5, dailyHRV, on=['Id','date'], how='left')
merged7 = pd.merge(merged6, fitbitBreathingRate, on=['Id','date'], how='left')
merged_final = pd.merge(merged7, fitbitSkinTemperature, on=['Id','date'], how='left')

In [12]:

merged_final_10 = merged_final[(merged_final['TotalMinutesWearTime'] >= 10*60)] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
merged_0_3000steps = merged_final[(merged_final['TotalMinutesWearTime'] == 0) & (merged_final['TotalSteps'] >= 3000)]

merged_final = pd.concat([merged_final_10, merged_0_3000steps])
merged_final = merged_final.sort_values(by=['Id', 'date']) 

#rename ID
merged_final['Id'] = [items[-3:] for items in merged_final['Id']]
merged_final['Id'] = [int(digit) for digit in merged_final['Id']]

Survey data, find sepecific coaching time

In [ ]:
survey_data['health_coach_survey_complete'].value_counts()

#select people who complete health coach session and the second follow-up
survey_data_clean = survey_data[survey_data['weeks_followup_survey_96ac_complete'] == 2] #87 participants

#select variables from survey
uses_vars_of_survey = ['record_id','current_status',
'demographics_age','demographics_sex','demographics_sexorient','demographics_ethnicity','demographics_immigration',
'demographics_race___1', 'demographics_race___2', 'demographics_race___3', 'demographics_race___4', 'demographics_race___5', 
'demographics_race___6', 'demographics_race___7', 'demographics_education', 'demographics_sorority',
'nervous_v1', 'down_v1', 'calm_v1', 'blue_v1', 'happy_v1',
'nervous_v2', 'down_v2', 'calm_v2', 'blue_v2', 'happy_v2',
'weeks_followup_survey_complete','weeks_followup_survey_96ac_complete',
'weeks_followup_survey_timestamp', 'weeks_followup_survey_96ac_timestamp'
]
select_survey = survey_data_clean[uses_vars_of_survey] #have the demongraphic information

#rename the name of record_id
select_survey.rename(columns = {"record_id":'Id'}, inplace=True)

#reframe the date time format
select_survey['weeks_followup_survey_timestamp'] = pd.to_datetime(select_survey['weeks_followup_survey_timestamp'], errors = 'coerce' )
select_survey['weeks_followup_survey_timestamp'] = select_survey['weeks_followup_survey_timestamp'].dt.date
select_survey['weeks_followup_survey_96ac_timestamp'] = pd.to_datetime(select_survey['weeks_followup_survey_96ac_timestamp'], errors = 'coerce' )
select_survey['weeks_followup_survey_96ac_timestamp'] = select_survey['weeks_followup_survey_96ac_timestamp'].dt.date

C:\Users\karri\AppData\Local\Temp\ipykernel_21020\608329705.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_survey.rename(columns = {"record_id":'Id'}, inplace=True)
C:\Users\karri\AppData\Local\Temp\ipykernel_21020\608329705.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_survey['weeks_followup_survey_timestamp'] = pd.to_datetime(select_survey['weeks_followup_survey_timestamp'], errors = 'coerce' )
C:\Users\karri\AppData\Local\Temp\ipykernel_21020\608329705.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [ ]:
select_survey['weeks_followup_survey_96ac_complete'].value_counts() #87 sample size, We only select the participants who completed the second follow-up

weeks_followup_survey_96ac_complete
2    87
Name: count, dtype: int64

In [308]:
#check the individual who did not complete second follow-up survey
select_survey[select_survey['weeks_followup_survey_96ac_complete'] == 0]['Id']

# 40      42
# 46      48
# 67      69
# 107    109dd
# 112    114
# 123    125
# 129    131
# 132    134
# 149    151

Series([], Name: Id, dtype: int64)

Merge Fitbit data with coaching time

In [309]:
fitbit_data = merged_final.copy()

#replace code 98 as 107 because the code of this participant is 107 in survey
fitbit_data.loc[fitbit_data['Id'] == 98, 'Id'] = 107

In [311]:
used_var = ['Id','date','TotalMinutesWearTime', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories', 'RestingHeartRate','TotalActiveZoneMinutes', 'FatBurnActiveZoneMinutes',
       'CardioActiveZoneMinutes', 'PeakActiveZoneMinutes', 'DailyRMSSD', 'DeepRMSSD',
       'AvgBreathsPerMinute', 'LightSleepAvgBreathsPerMinute','DeepSleepAvgBreathsPerMinute', 'REMSleepAvgBreathsPerMinute',
       'TotalMinutesAsleep_x', 'TotalTimeInBed_x', 'TotalTimeAwake', 'TotalMinutesLight', 
       'TotalMinutesDeep','TotalMinutesREM', 'NightlyRelative', 'Value'] #delete 'Above', 'Below', Custom Zone; NightlyRelative represents ski temparature

used_fitbit_data = fitbit_data[used_var]

used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep','TotalTimeInBed_x':'TotalMinutesinbed',
                                   'TotalTimeAwake': 'TotalMinutesAwake',
                                   'Value': 'HR',
                                   'NightlyRelative': 'skin_temparature'}, 
                                   inplace = True)

#select participants matched with survey
all_survey_id = select_survey['Id'].unique()
used_fitbit_data = used_fitbit_data[used_fitbit_data['Id'].isin(all_survey_id)]

#add another column Fitbit data
used_fitbit_data.insert(loc = 2, column = 'survey_date', value = np.nan)

C:\Users\karri\AppData\Local\Temp\ipykernel_21020\1194012362.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep','TotalTimeInBed_x':'TotalMinutesinbed',


In [351]:
# impute time sequence data
time_sequency = used_fitbit_data.copy()
all_fitbit_id = time_sequency['Id'].unique()
new_data_frame = pd.DataFrame()

for i in range(len(all_fitbit_id)):

    fitbit_id = all_fitbit_id[i]

    sorted_individual = time_sequency[time_sequency['Id'] == fitbit_id].sort_values(by = 'date')
    
    #outcome variable
    individual_survey = select_survey[select_survey['Id'] == fitbit_id]
    followup1_time = individual_survey['weeks_followup_survey_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_timestamp'].empty else pd.NaT
    followup2_time = individual_survey['weeks_followup_survey_96ac_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_96ac_timestamp'].empty else pd.NaT
    
    
    start_date = sorted_individual['date'].iloc[0]
    end_date = followup2_time
    
    full_date_range = pd.date_range(start=start_date, end=end_date)

    sorted_individual.set_index('date', inplace=True)
    
    sorted_individual_reindexed = sorted_individual.reindex(full_date_range).reset_index().fillna(0) #reset the wrong index
    sorted_individual_reindexed['Id'] = fitbit_id
    sorted_individual_reindexed.rename(columns = {'index':'date'}, inplace = True)

    new_data_frame = pd.concat([new_data_frame, sorted_individual_reindexed])

In [ ]:
def assign_survey_date(row, followup1_time, followup2_time):
    time = row['date']
    
    if pd.isna(followup2_time):
        if pd.notna(time) and time < followup1_time:
            return 1
        if pd.notna(time) and time == followup1_time:
            return 2
    
    elif pd.notna(followup2_time):
        if pd.notna(time) and time < followup1_time:
            return 1
        if pd.notna(time) and time == followup1_time:
            return 2
        if pd.notna(time) and followup1_time < time < followup2_time:
            return 3
        if pd.notna(time) and time == followup2_time:
            return 4
    
    return None

In [391]:
def assign_survey_date(time, followup1_time, followup2_time):
    if time < followup1_time:
        return 1
    elif time == followup1_time:
        return 2
    elif followup1_time < time < followup2_time:
        return 3
    elif time == followup2_time:
        return 4
    
    return None

In [397]:
new_used_fitbit_data = new_data_frame.copy()

for i in range(len(all_fitbit_id)):

    fitbit_id = all_fitbit_id[i]
    #print(fitbit_id)

    #outcome variable
    individual_survey = select_survey[select_survey['Id'] == fitbit_id]
    
    followup1_time = individual_survey['weeks_followup_survey_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_timestamp'].empty else pd.NaT
    #print(followup1_time)
    followup2_time = individual_survey['weeks_followup_survey_96ac_timestamp'].iloc[0] if not individual_survey['weeks_followup_survey_96ac_timestamp'].empty else pd.NaT
    #print(followup2_time)
    
    #predictors
    individual_fitbit = new_used_fitbit_data[new_used_fitbit_data['Id'] == fitbit_id].copy()

    individual_fitbit['survey_date'] = individual_fitbit.apply(
        lambda row: assign_survey_date(row['date'].date(), followup1_time, followup2_time), axis=1
        )
    
    new_used_fitbit_data.loc[new_used_fitbit_data['Id'] == fitbit_id, 'survey_date'] = individual_fitbit['survey_date'] #replace with new define

In [400]:
#Add coach variable
fitbit_coach = new_used_fitbit_data.copy()

#fitbit_coach.insert(loc = 3, column = 'coach_date', value = np.nan)
coach_time['Health Coach Session'] = pd.to_datetime(coach_time['Health Coach Session'])

all_fitbit_id = time_sequency['Id'].unique()

for ID in all_fitbit_id:
    individul_person = fitbit_coach[fitbit_coach['Id'] == ID]
    individual_coach_time = coach_time[coach_time['ID'] == ID]['Health Coach Session'].iloc[0]

    for date in individul_person['date']:
        if date == individual_coach_time:
            fitbit_coach.loc[(fitbit_coach['Id'] == ID) & (fitbit_coach['date'] == date), 'coach_date'] = 1
        else:
            fitbit_coach.loc[(fitbit_coach['Id'] == ID) & (fitbit_coach['date'] == date), 'coach_date'] = 0

Merge fitbit data with survey data

In [415]:
# The average of the five responses were used for data analysis
select_survey['mental_health_1'] = select_survey[['nervous_v1', 'down_v1', 'calm_v1','blue_v1', 'happy_v1']].mean(axis = 1)
select_survey['mental_health_2'] = select_survey[['nervous_v2', 'down_v2', 'calm_v2','blue_v2', 'happy_v2']].mean(axis = 1)

C:\Users\karri\AppData\Local\Temp\ipykernel_21020\2954688467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_survey['mental_health_1'] = select_survey[['nervous_v1', 'down_v1', 'calm_v1','blue_v1', 'happy_v1']].mean(axis = 1)
C:\Users\karri\AppData\Local\Temp\ipykernel_21020\2954688467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_survey['mental_health_2'] = select_survey[['nervous_v2', 'down_v2', 'calm_v2','blue_v2', 'happy_v2']].mean(axis = 1)


In [416]:
#recode demongraphic variable
all_fitbit_id = time_sequency['Id'].unique()
var = ['demographics_race___1','demographics_race___2','demographics_race___3','demographics_race___4',
       'demographics_race___5','demographics_race___6','demographics_race___7']
recode_demographic_variable = select_survey[var]
recode_demographic_variable['demographic'] = np.nan
recode_demographic_variable['Id'] = select_survey['Id']

#1: demographics_race___1 = 1
#2: demographics_race___2 = 1
#3: demographics_race___3 = 3
#4: demographics_race___4 = 4
#5: demographics_race___5 = 5
#6: demographics_race___6 = 6
#7: demographics_race___7 = 7
#8: multiple_race = 8

# Recoding logic
for i in all_fitbit_id:
    row = recode_demographic_variable[recode_demographic_variable['Id'] == i][var]
    # Check if there are multiple races selected
    if row.sum(axis=1).iloc[0] > 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 8  # Multiple races
    elif row['demographics_race___1'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 1
    elif row['demographics_race___2'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 2
    elif row['demographics_race___3'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 3
    elif row['demographics_race___4'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 4
    elif row['demographics_race___5'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 5
    elif row['demographics_race___6'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 6
    elif row['demographics_race___7'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 7

select_survey['demographics_race'] =  recode_demographic_variable['demographic']

C:\Users\karri\AppData\Local\Temp\ipykernel_21020\965030629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recode_demographic_variable['demographic'] = np.nan
C:\Users\karri\AppData\Local\Temp\ipykernel_21020\965030629.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recode_demographic_variable['Id'] = select_survey['Id']
C:\Users\karri\AppData\Local\Temp\ipykernel_21020\965030629.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [417]:
used_variables = ['Id', 'current_status','demographics_age','demographics_sex','demographics_sexorient',
                  'demographics_ethnicity', 'demographics_immigration','demographics_race',
                  'demographics_education', 'demographics_sorority']
select_data = select_survey[used_variables]

In [419]:
#merge Fitbit data with survey data
fitbit_survey = pd.merge(fitbit_coach, select_data, on = 'Id', how = 'left')

In [428]:
# Transpose the entire DataFrame
fitbit_survey_transfer = fitbit_survey.copy()


Convert horizontal to vertical data

In [ ]:
#convert horizontal to vertical
final_data_frame = pd.DataFrame()

for i1 in range(len(all_fitbit_id)):
    fitbit_id = all_fitbit_id[i1]
    select_data = fitbit_survey_transfer[fitbit_survey_transfer['Id'] == fitbit_id]

    len_select_data = len(select_data)

    process_series = pd.Series(dtype = 'object')

    for i2 in range(len_select_data):
        row_select_data = select_data.iloc[i2].copy()
        row_select_data.index = [str(col) + '_' + str(i2) for col in row_select_data.index] #revise the name
        process_series = pd.concat([process_series, row_select_data])

    trans_process_series = pd.DataFrame(process_series).T

    final_data_frame = pd.concat([final_data_frame, trans_process_series], axis = 0, ignore_index=True)

final_data_frame = final_data_frame.set_index(['Id_0'])#set ID and date as index
final_data_frame = final_data_frame.drop(final_data_frame.filter(regex = 'Id').columns, axis = 1) #delete column with the same format
final_data_frame.reset_index(inplace=True) #reset_index
final_data_frame.rename(columns = {'Id_0':'Id'}, inplace = True) #renamei d

In [ ]:
#combine with mental health dataset
mental_health_data = select_survey[['Id','mental_health_1','mental_health_2']]
final_data_mental_health = pd.merge(final_data_frame, mental_health_data, on = 'Id', how = 'inner')

In [464]:
fitbit_survey.to_csv('final_data_mental_health.csv')

Make the participants have the same time frame

In [324]:
time_difference = pd.to_datetime(fitbit_survey['enddate']) - fitbit_survey['startdate']
mean_total_days = np.mean(time_difference).days
columns = final_data_frame.columns
add_column = list(fitbit_survey.columns[-22:])
add_column.append('Id')

suffix_range_rest = [f"_{i}" for i in range(mean_total_days)]
set_all_columns_the_same = [col for col in columns if any(col.endswith(suffix) for suffix in suffix_range_rest)]

reorganize_data = pd.DataFrame()


In [325]:
num_i = len(time_difference)

for i in range(num_i):
    i = i
    difference_i = time_difference[i]
    n_days = time_difference[i].days

    if n_days <= mean_total_days:
        suffix_range = [f"_{i}" for i in range(mean_total_days)]
        select_fitbit_columns = [col for col in columns if any(col.endswith(suffix) for suffix in suffix_range)]
        used_data = pd.DataFrame(fitbit_survey.iloc[i]).T[select_fitbit_columns]

    else:
        nan_day = []
        suffix_range_new = [f"_{i}" for i in range(mean_total_days+1, n_days)]

        for suffixs in suffix_range_new:
            suffix_columns = [col for col in columns if col.endswith(suffixs)]
            
            #print(len(suffix_columns))
            suffix_data = fitbit_survey.iloc[i][suffix_columns]

            if suffix_data.isna().all():
                #print(True)
                nan_day.append(suffixs)

        #select data after 65
        filter_suffix_range = [item for item in suffix_range_new if item not in nan_day]

        if len(filter_suffix_range) >= mean_total_days:
            new_filter_suffix_range = filter_suffix_range[-mean_total_days:]
            columns_65 = [col for col in columns if any(col.endswith(suffix) for suffix in new_filter_suffix_range)]
            columns_65_data = pd.DataFrame(fitbit_survey.iloc[i]).T[columns_65]
            used_data = columns_65_data

        else:
            #select data before 65
            #number of columns before 65
            n_col = mean_total_days-len(filter_suffix_range)
            start_number = mean_total_days-n_col
            end_number = mean_total_days

            suffix_range = [f"_{i}" for i in range(start_number+1, end_number+1)]
            select_fitbit_columns = [col for col in columns if any(col.endswith(suffix) for suffix in suffix_range)]
            before_65_data = pd.DataFrame(fitbit_survey.iloc[i]).T[select_fitbit_columns]

            #select dat aafter 65

            after_65_columns = [col for col in columns if any(col.endswith(suffix) for suffix in filter_suffix_range)]
            after_65_data = pd.DataFrame(fitbit_survey.iloc[i]).T[after_65_columns]

            used_data = pd.concat([before_65_data, after_65_data], axis = 1) #combine two

        used_data.columns = set_all_columns_the_same

    #append survey data
    survey_data = pd.DataFrame(fitbit_survey.iloc[i]).T[add_column]

    new_fitbit_survey = pd.concat([used_data, survey_data], axis = 1)
        
    reorganize_data = pd.concat([reorganize_data, new_fitbit_survey], axis = 0, ignore_index=True) #

In [327]:
reorganize_data.to_csv('final_survey_fitbit_time_series_v2.csv')